## Importing Libraries

We will add every library only when it is needed

In [2]:
import os

---

## Importing Data for Reading

We have two folders named (Images, Annotations)

Images Folder:
It contains only images, 7,393 images 

Annotations Folder:
It conatins two folders names (xmls, trimaps) and three files (trainval.txt, test,txt, list,text)

xmls is bounding boxes which is used for object detection

trimaps is segements which is used for image segmentation

the other three files explain the data and categorize them so the are used for classification 

We will use the three files now for our classification task then we will us trimaps for segmentation.

In [3]:
# Adding the path of the data of classification
base_path = r"C:\Users\omara\Downloads\Multiclass Classification & Semantic Segmentation of Pet Breeds\data\oxford-iiit-pet"  
images_dir = os.path.join(base_path, "images")
annotations_dir = os.path.join(base_path, "annotations")

# Reading the annotation files used for classification
trainval_file = os.path.join(annotations_dir, "trainval.txt")
test_file = os.path.join(annotations_dir, "test.txt")
list_file = os.path.join(annotations_dir, "list.txt")

---